In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

# --- Caminho para os resultados ---
base_path = "agricultural_csp/results"
csv_files = glob.glob(os.path.join(base_path, "*.csv"))

# --- Leitura e concatenação de todos os resultados ---
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

# --- Extração da instância (antes do último "_<strategy>") ---
df["Instance"] = df["Run Name"].apply(
    lambda x: "_".join(str(x).split("_")[:-1]) if isinstance(x, str) else np.nan
)

# --- Cria pivot table com valores finais por instância e solver ---
pivot = df.pivot_table(
    index="Instance",
    columns="Strategy",
    values="Final Obj"
)

# --- Remove instâncias com dados faltantes ---
pivot = pivot.dropna()

# --- Calcula os performance ratios ---
ratios = pivot.divide(pivot.min(axis=1), axis=0)

# --- Define o eixo tau (intervalo do fator de desempenho) ---
taus = np.linspace(1, ratios.max().max(), 200)

# --- Plota o gráfico ---
plt.figure(figsize=(10, 6))
num_instances = len(pivot)

label_map = {
    'first': 'Busca Tabu - First Improving',
    'best': 'Busca Tabu - Best Improving'
}

for solver in ratios.columns:
    y = [(ratios[solver] <= tau).sum() / num_instances for tau in taus]
    plt.plot(taus, y, drawstyle='steps-post', label=label_map.get(solver, solver), linewidth=2)

plt.title('Performance Profile (Valor da Função Objetivo)', fontsize=14)
plt.xlabel('Razão de performance τ', fontsize=12)
plt.ylabel('Proporção de Instâncias Resolvidas', fontsize=12)
plt.xlim(left=1)
plt.ylim(0, 1.05)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='', loc='lower right')
plt.tight_layout()
plt.savefig("performance_profile.png", dpi=300, bbox_inches='tight')
plt.show()


ParserError: Error tokenizing data. C error: Expected 20 fields in line 402, saw 21
